In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

import ipywidgets as widgets
from IPython.display import display
from datetime import date, timedelta

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default = 'notebook'

import ptm_lib as ptm
%load_ext autoreload
%autoreload 2

# import pandas_datareader as pdr
# from pandas_datareader import wb
# import quandl
# import matplotlib.pyplot as plt
# from matplotlib import colors

In [2]:
def sma(fig, df, col_name, win_size):  
    subset = df.copy()
#     fig.add_trace(go.Scatter(x=subset.index, y=subset[col_name]))
    subset['rolling_avg'] = subset[col_name].rolling(window=win_size).mean()
    fig.add_trace(go.Scatter(x=subset.index, y=subset['rolling_avg'],mode="lines", name=str(win_size)+'-SMA'))

    fig.update_layout(title_text='<b>'+'SMA'+'-'+col_name+'<b>', title_x=0.5)
    fig.update_layout(margin=dict(l=0, r=0, t=30, b=0))
#     fig.show()

In [7]:
from_date = date(1928, 1, 1) 
today = date.today()
tickers = ['AMZN']
interval='d'
ticker = tickers[0]
win_size = 40

df = ptm.load_data(tickers, from_date, today, interval)
df.rename(columns={'Adj Close':ticker}, inplace=True)
# df.fillna(method='bfill')
df

[*********************100%%**********************]  1 of 1 completed


,AMZN
Date,
1997-05-15,0.097917
1997-05-16,0.086458
1997-05-19,0.085417
1997-05-20,0.081771
1997-05-21,0.071354
...,...
2024-04-09,185.669998
2024-04-10,185.949997
2024-04-11,189.050003


In [8]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df[ticker],name=ticker))
sma(fig, df, ticker, 20)
sma(fig, df, ticker, 60)
sma(fig, df, ticker, 120)
sma(fig, df, ticker, 250)
fig.show()

In [10]:
def rsi(fig, df, col_name, win_size=20):  
    subset = df.copy()
    # Calculate price changes
    subset['PriceChange'] = df[col_name].diff()
    # Calculate gains and losses
    subset['Gain'] = subset['PriceChange'].apply(lambda x: x if x > 0 else 0)
    subset['Loss'] = subset['PriceChange'].apply(lambda x: -x if x < 0 else 0)
    # Calculate average gains and losses
    subset['AvgGain'] = subset['Gain'].rolling(window=win_size, min_periods=1).mean()
    subset['AvgLoss'] = subset['Loss'].rolling(window=win_size, min_periods=1).mean()
    # Calculate RS and RSI
    subset['RS'] = subset['AvgGain'] / subset['AvgLoss']
    subset['RSI'] = 100 - (100 / (1 + subset['RS']))
    
    fig.add_trace(go.Scatter(x=subset.index, y=subset['RSI'],mode="lines", name=str(win_size)+'-RSI'))

    fig.update_layout(title_text='<b>'+'RSI'+'-'+col_name+'<b>', title_x=0.5)
    fig.update_layout(margin=dict(l=0, r=0, t=30, b=0))    
    
#     return subset[['RSI']]


In [11]:
fig = go.Figure()
# fig.add_trace(go.Scatter(x=df.index, y=df[ticker],name=ticker))
rsi(fig, df, ticker, 20)
fig.show()